Jak przewidywać ceny na giełdzie?
Użyjemy do tego RNN traktując wykres cen akcji jako szeregi czasowe.
Posłużymy się najmocniejszym z poznanych jednostek rekurencyjnych - LSTM.

In [ ]:
#!pip install tensorflow-gpu
import tensorflow as tf


print(tf.__version__)

In [ ]:
from tensorflow.keras.layers import Input, LSTM, GRU, SimpleRNN, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
################ Część I ##############################
# pobieramy dane (dzienne ceny akcji starbucks od lutego 2013 do lutego 2018)
df = pd.read_csv('https://raw.githubusercontent.com/jgrynczewski/rnn/master/sbux.csv')

In [ ]:
df.head()
# Kiedy myślimy o cenie akcji mamy zazwyczaj w głowie jedną cenę/liczbę na dzień.
# Więc moglibysmy myślec o tym jako o jednowymiarowym szerego czasowym.
# Ale tu mamy wiele kolumn. Czym one są?

# Zwrócmy uwagę na skalę. Ceny są podawane w dziesiątkach, podczas gdy wolumeny
# w milionach

In [ ]:
df.tail()

Widzimy, że przez 5 lat ceny akcji Starbucksa podwoiły się, co jest
dobrą informacją dla wszystkich akcjonariuszy, posiadaczy akcji Starbucks

In [ ]:
# Zacznijmy z jednowymiarowym szeregiem czasowym. Skupmy się na cenie zamknięcia.

series = df['close'].values.reshape(-1, 1)
# Bierzemy wszystkie ceny zamknięca jako np array i zmianiamy wymiar na Nx1
# Musimy to zrobić, ponieważ następny krok to zastosowanie StandardScalar z scikit-learn
# po to, żeby dane były ustandaryzowane.

In [ ]:
# Normalizacja
scaler = StandardScaler()
scaler.fit(series[:len(series)//2])  # fit robimy tylko na
# pierwszej cześci szeregu (bo nie chcemy zanieczyścić danych testowych podczas treningu)
series = scaler.transform(series).flatten()  # z kolei transform robimy na 
# całym zbiorze.
# Dodatkowo, ponieważ będziemy okienkowali szereg musielibysmy zrobić kilka obliczeń,
# żeby określić dokładnie granicę pomiędzy zbiorem treningowym i testowym. My sobie
# uprościmy mówiąc, że ta granica jest dokładnie w połowie.
# Na koniec spłaszczamy szereg, żeby dostać wektor długości N.

In [ ]:
# Okienkujemy nasz szereg.
# Koncepcyjnie nie różni się to niczym od tego co robiliśmy z sinusem.
# Jedyną różnicą są tu dane. Sposób postępowania jest niezmienny.
T = 10
D = 1
X = []
Y = []

for t in range(len(series) - T):
  x = series[t:t+T]
  X.append(x)
  y = series[t+T]
  Y.append(y)

X = np.array(X).reshape(-1, T, 1)  # N x T x D
Y = np.array(Y)
N = len(X)

print(f"X.shape {X.shape}, Y.shape {Y.shape}")

In [ ]:
# Na sinusach poza LSTM testowaliśmy model autoregresyjny oraz SimpleRNN
# Tutaj popatrzmy już na LSTM
i = Input(shape=(T, 1))
x = LSTM(5)(i)
x = Dense(1)(x)
model = Model(i, x)
model.compile(
    loss='mse',
    optimizer=Adam(lr=0.1),
)

# train the RNN  - zbiór treningowy, pierwsza połowa. Zbiór testowy, druga połowa
r = model.fit(
    X[:-N//2],
    Y[:-N//2],
    epochs=80,
    validation_data=(X[-N//2:], Y[-N//2:]),
)

In [ ]:
# widzimy, że koszt spada co jest obiecujące
# to oznacza, że LSTM jest w stanie przewidzieć coś blisko kolejnej próbki
# szeregu czasowego.
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
# jednokrokowa predykcja
outputs = model.predict(X)
print(outputs.shape)
predictions = outputs[:, 0]

plt.plot(Y, label='targets')
plt.plot(predictions, label='predictions')
plt.legend()
plt.show()

# Wynik jest bardzo zgodny, ale dlaczego nie jest to dla nas 
# zbytnio pomocne. Bo tak naprawdę przewidujemy tylko cenę na kolejny
# dzień, nie jest to informacja na podstawie której można podjąc
# sensowne decyzje dotyczące sprzedaży/kupna akcji.
# Chcielibyśmy przewidzieć dalekie tendencje, górki i dołki.
# Może chociaż na dwa trzy dni do przodu. Popatrzmy

In [ ]:
# Wielokrokowa predykcja
validation_target = Y[-N//2:]
validation_predictions = []

# ostatni input 
last_x = X[-N//2] 

while len(validation_predictions) < len(validation_target):
  p = model.predict(last_x.reshape(1, T, 1))[0, 0]  # 1x1 array -> scalar

  validation_predictions.append(p)

  last_x = np.roll(last_x, -1)
  last_x[-1] = p

plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend() # widzimy, że to już nie wygląda dobrze. Kiedy robimy prognoze wielokrokową
# wszystko co dostajemy to prosta linia.

# Czyli nasz model wcale nie jest taki świetny.
# W zasadzie nie robi wiele. 

In [ ]:
################ Część II ##############################
# Lekcja 1 - jednokrokowa prognoza ceny jest myląca. Poza tym
# współczesne modele w ogóle nie patrzą na cenę akcji.
# To na co patrzą, to co próbują przewidzieć to zwrot akcji (stock return)
# Zwrot definiujemy
# R = V_final - V_initial / V_initial

# To ta sama formuła, co wtedy kiedy przechodząc obok sklepu,
# widzimy przecena. np. 20%. Co to oznacza ?
# Powiedzmy, że coś kosztowało 100. Po przecenie kosztuje 80

# 80 - 100 / 100 = -0.2 = -20% (czyli procentowy spadek/wzrost ceny)

# Tak więc kiedy finansowi inżynierowie prognozują akcje, przeważnie
# patrzą na pewną formę zwrotu, a nie na aktualną cenę.

# Wrócmy do kodu.

# Skoro pracujemy z Pandas ta zmiana będzie dość prosta. Wystarczy tylko wiedzieć
# jakiej funkcji użyć.
# Jak zwykle chcemy procesować nasze dane w postaci wektorów (tzn. wykonujemy
# operacje na wszystkich kolumnach za jednym razem)

# 1. Tworzymy nową kolumnę PrevClose taką, która jest przesuniętą o 1 ceną 
# zamknięcia poprzedniego dnia. Tak żeby wczorajsza cena zamknięcia (PrevClose)
# była obok dziejszej ceny zamknięcia.
df['PrevClose'] = df['close'].shift(1)

# Teraz wygląda to tak
# close / prev lose
# x[2] x[1]
# x[3] x[2]
# x[4] x[3]
# ...
# x[t] x[t-1]

In [ ]:
df.head() # widzimy, że pierwsza wartość PrevClose to Nan,
# ponieważ nie mamy ceny sprzed pierwszego dnia. Nie jest to
# dla nas problem, ponieważ będziemy budować RNN, które bierze
# pierwsze 10 dni, żeby przewidzieć 11. Czyli pierwszą wartość
# którą będziemy przewidywać to wartość 11 (na podstawie 
# 10 poprzednich). Dopóki ten NaN nie jest celem, nie musimy się 
# tym przejmować.

In [ ]:

# Liczymy zwroty - dodatkowa kolumna
# x[t] - x[t-1] / x[t-1]
df['Return'] = (df['close'] - df['PrevClose']) / df['PrevClose']

In [ ]:
df.head()

In [ ]:
# Popatrzmy na rozkład zwrotów. Widzimy, że najczęściej jest to 0.
# Widzimy, że zwroty są bardzo małymi wartościami. 
# Możemy chcieć je znormalizować. Zróbmy to.
df['Return'].hist()

In [ ]:
series = df['Return'].values[1:].reshape(-1, 1)  # Po pierwsze, znowu robimy
# z tego macierz Nx1, bo użyjemy StandardScaler.

# Normalizacja zwrotów
scaler = StandardScaler()
scaler.fit(series[:len(series) // 2])  # Nie normalizujemy danych testowych.
# Normalizacja ma na celu jedynie zmniejszyć złożoność obliczeniową.
# Na znormalizowanych danych obliczenia są szybsze.
# fit na pierwszej połowie
series = scaler.transform(series).flatten()  # transform na całym szeregu
# i na koniec flatten do wektora długości N.

In [ ]:
### te same kroki, ale tym razem sekwencją nie są kolejne
# ceny akcji, ale kolejne zwroty
T = 10
D = 1
X = []
Y = []

for t in range(len(series) - T):
  x = series[t:t+T]
  X.append(x)
  y = series[t+T]
  Y.append(y)

X = np.array(X).reshape(-1, T, 1)  #  N x T x D
Y = np.array(Y)
N = len(X)
print(f"X.shape {X.shape}, Y.shape {Y.shape}")

In [ ]:
# LSTM, jedyną różnicą są tu dane. Kod ten sam.
i = Input(shape=(T, 1))
x = LSTM(5)(i)
x = Dense(1)(x)
model = Model(i, x)
model.compile(
    loss='mse',
    optimizer=Adam(learning_rate=0.01),
)


r = model.fit(
    X[:-N//2],
    Y[:-N//2],
    epochs=80,
    validation_data=(X[-N//2:], Y[-N//2:])
)

In [ ]:

plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

# Co widzimy ?
# Modelowi jest znacznie trudniej nauczyć się cokolwiek.
# Koszt z każdą iteracją spada, ale na zbiorze walidacyjnym rośnie.
# Innymi słowy, model coraz lepiej dopasowuje się do szumu.

In [ ]:
# Jednokrokowa predykcja
outputs = model.predict(X)
print(outputs.shape)
predictions = outputs[:, 0]

plt.plot(Y, label='targets')
plt.plot(predictions, label='predictions')
plt.legend()
plt.show()

# Na podstawie tego wykresu trudno jest ocenić, czy prognozy są
# poprawne, ale znając kolejne wartościfunkcji kosztu domyślamy się, że nie.
# Można to uruchomić lokalnie i zoomować sprawdzając.

In [ ]:
# Wielokrokowa predykcja
validation_target = Y[-N//2:]
validation_predictions = []

last_x = X[-N//2] 

while len(validation_predictions) < len(validation_target):
  p = model.predict(last_x.reshape(1, T, 1))[0, 0]  # 1x1 array -> scalar

  validation_predictions.append(p)

  last_x = np.roll(last_x, -1)
  last_x[-1] = p

plt.plot(validation_target, label='forecast target')
plt.plot(validation_predictions, label='forecast prediction')
plt.legend()

# Znów mamy do czynienia z sytaucją kiedy model potrafi tylko kopiować
# jedną wartość w kółko. 

In [ ]:
################### Część III #########################
# Model 3 - Zbudujmy lepszy model
# W modelu 2 zdecydowaliśmy się na regresję.
# Popatrzmy na to zagadnienie z perspektywy klasyfikacji.
# Użyjemy wszystkich informacji - cena otwarcia, najwyższ, najniższa, zamknięcia oraz wolumen.
# A na podstawie tych informacji będziemy starali się przewidzieć czy cena pójdzie
# w górę, czy w dół.

# Klasyfikajca: w górę ?

# Czyli będziemy mieli input TxD (gdzie T=10, D=5 - liczba cech)
# No i na jego podstawie nie będziemy próbowali przewidzieć zwrotu, zamiast 
# tego zamienimy to w najprostsze możliwe zadanie. Spróbujemy przewidzieć, 
# czy cena pójdzie w górę, czy w dół - klasyfikajca binarna.

# W ogólności w klasyfikacji łatwiej otrzymać satysfakcjonujące wyniki niż w regresji.
# W regresji przewidujemy ciągłe wartości. Nie możesz być za wysoko czy za nisko,
# musisz być dokładnie w tej wartości. Klasyfikacja jest prostsza. Zwłaszcza
# klasyfikacja binarna. Nie trzeba przewidywać dokładnej wartości. Wystarczy
# etykieta. W tym przypadku mamy tylko dwie wartości: do góry, do dołu.
# I zazwyczaj to jest właśnie to co nas interesuje w przypadku akcji.

In [ ]:
# Załadujmy wszystkie dane do numpy arrays

input_data = df[['open', 'high', 'low', 'close', 'volume']].values
targets = df['Return'].values  # target będziemy określali na podstawie zwrotu.

In [ ]:
# Ustawmy N, T i D
T = 10  # liczba elementów na podstawie których prognozujemy następną wartość
D = input_data.shape[1]  # liczba kolumn danych wejściowych
N = len(input_data) - T # N jest trochę podchwytliwe. Długość szeregu - okienko.
# Liczyliśmy to kilka razy.
# (na przykład jeżeli T=10 i mamy tylko 11 próbek wtedy mamy N=1)

In [ ]:
# normalizacja - tym razem jest to szczególnie istotne ponieważ
# kolumna wolumenu ma bardzo duże liczby, w porównaniu do wartości w kolumnach
# z ceną.
# Tym razem zrobimy to trochę łatwiejsze dla naszego modelu (więcej treningu).
# Zamiast ustawiać pierwszą połowę jako zbiór uczący, a drugą jako testowy,
# zróbmy 2/3 zbiorem treningowym, a pozostałe 1/3 zbiorem testowym.
Ntrain = len(input_data) * 2 // 3
scaler = StandardScaler()
scaler.fit(input_data[:Ntrain + T])
input_data = scaler.transform(input_data)

In [ ]:
# Tworzymy zbiór treningowy
# X_train jest rozmiaru Ntrain x T x D
# Y_train jest rozmiaru Ntrain
X_train = np.zeros((Ntrain, T, D))
Y_train = np.zeros(Ntrain)

# Wypełniamy dane
for t in range(Ntrain):
  X_train[t, :, :] = input_data[t:t+T] # X to input_data od t do t+T
  Y_train[t] = (targets[t+T] > 0)  # Y to informacja o tym, czy zwrot w t+T był dodatni. 

In [ ]:
# Tworzymy zbiór uczący
# Ntest = N - Ntrain
X_test = np.zeros((N - Ntrain, T, D))
Y_test = np.zeros(N - Ntrain)

for u in range(N - Ntrain):
  # u of 0 do (N - Ntrain)
  # t od Ntrain do N
  t = u + Ntrain # indexujemy oryginalny zbiór danych, czyli musimy
  # zachować offset Ntrain. Uzywamy t do indeksowania oryginalnego
  # zbiou, a u do indeksowania X_test i Y_test.
  X_test[u, :, :] = input_data[t:t+T]
  Y_test[u] = (targets[t+T] > 0)

In [ ]:
# pamiętamy, że tym razem robimy klasyfikację bianarną
# więc ostatnia warstwa to ma jeden węzeł i funkcję aktywacji - sigmoid.
# loss to binary_crossentropy
i = Input(shape=(T, D))
x = LSTM(50)(i)
x = Dense(1, activation='sigmoid')(x)
model = Model(i, x)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [ ]:
# trenujemy
r = model.fit(
    X_train,
    Y_train,
    batch_size=32,
    epochs=300,
    validation_data=(X_test, Y_test)
)

In [ ]:
# koszt
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# Widzimy, że koszt zbioru treningowego spada lekko, ale koszt na zbiorze 
# walidującym mocno wzrasta. To mówim nam, że model znów przeucza się szumu. 

In [ ]:
# Jak popatrzymy na dokładność widzimy to samo.
# Dokładnośc na zbiorze treningowym wzrasta, ale dokładnośc na zbiorze
# walidacyjnym nie.
# Można zadać sobie pytanie dlaczego dokładnośc na zbiorze testowym nie idzie w dół,
# tylko pozostaje na 0.5. Dla klasyfikacji binarnej 0 nie jest wcale najgorszą 
# dokładnością. Jeżeli twoja dokładność wynosi 0 oznacza to, że wystarczy tylko 
# odwrócić prognozy i będziemy mieli 100 % dokładność.
# Dla klasyfikacji bianrnej najgorszą dokładnością jest 0.5. 0.5 oznacza, że
# model zachowuje się jakby zgadywał, wybierał losowo, rzucał monetą. Dlatego
# kiedy model się przeucza dokładnośc modelu na zbiorze testowym pozostaje 0.5.

plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

I to tyle. Aktualnie nie istniej model, który potrafiłby poprawnie przewidywać ceny akcji. 

Zrobiliśmy trzy podejścia do tematu:
1. próbowaliśmy przewidzieć przyszłą cenę akcji na podstawie przeszłej ceny za pomocą modelu LSTM ('zadziałało' tylko dla jednokrokowej prognozy). Model zachowywał się tak jakby pamiętał tylko poprzednią wartość i na jej podstawie prognozował.
2. próbowaliśmy przewidzieć zwrot na podstawie poprzednich zwrotów. W tym przkładzie nie udało się uzyskać dużego spadku funkcji kosztu.
3. użyliśmy wszystkich dostępnych danych i klasfikatora binarnego.

Nawet próba rozwiązania zagadnienia w możliwe najprostszym sformułowaniu dała nie lepsze rezultaty niż losowe zgadywanie wyniku.

Lekcja?
Nie ma mowy, żeby dwa pierwsze modele zadziałały. Skoro nie potrafimy przewidzieć tego czy cena akcji pójdzie w górę czy w dół jak moglibyśmy przewidzieć numeryczną wartość zwroty czy ceny.

Bądź podejrzliwy kiedy ktoś mówi o modelu dokładnie przewidującym ceny akcji.

Podstawowym probleme w przewidywaniu cen akcji na podstawie cen akcji jest to, że nie uwzględniamy danych świata rzeczywistego, które mają wpływ na te ceny. Cena jest skorelowana z poprzednią wartością, ale znacznie silniej może być skorelowana z jakimś wydarzeniem na świecie. Nawet jeżeli zaczniemy uwzględniać różne wydarzenia napotkamy ogromne trudności związane z chaotycznym charakterem zjawiska.
Poza tym na cenę akcji wpływ mają też takie rzeczy jak nastroje inwestorów czy wizerunek firmy w mediach. Nawet bez znajomości DL, powinno wzbudzać nasze podejrzenie informacje o kursach, metodach przewidywania cen akcji na podstawie danych historycznych.

Stwierdzenie, że na podstawie danych historycznych ktoś jest w stanie przewidzieć ceny akcji jest równoznaczne ze stwierdzeniem, że na podstawie historycznych cen akcji ktoś jest w stanie przewidzieć dochodzenie dotyczące facebooka, wysłanie głupiego tweeta przez Elona Muska czy opracowanie nowej rakiety.

Ale nie oznacza to, że LSTM nie są dobre.
LSTM udowadniają swoją skuteczność w takich dziedzinach jak modelowanie języka i tłumaczenie maszynowe. Widzieliśmy już jak LSTM może być użyte do klasyfikacji zdjęć.

Ważne, żeby umieć rozróżnić przewidywanie wartości na wiele kroków do przodu od przewidywania jednego kroku do przodu. Przewidywanie jeden krok do przodu nie jest błędem o ile stosuje się go w odpowiednim miejscu, a nie jako model przewidujący na wiele kroków do przodu.
